# iBioFoundry #2 - Transformation Adder - OT2
author: Camillo Moschner | version: 2.0 | date: 23.01.2022 | license = 

# 1- Metadata

In [1]:
metadata = {
    'protocolName': 'iBF_2_T&SOCadder',
    'author': 'Camillo Moschner <cm967@cam.ac.uk> / <camillo.moschner@gmail.com>',
    'description': 'Automated Adding of Assembled DNA Constructs, and after heat-shock, SOC media to Chemically Competent Cells using OT2',
    'apiLevel': '2.12',
    'Date': '10.05.2022',
    'pipette_configuration':{'left':'p20_single_gen2',
                             'right':'p300_single_gen2'}
    }

## Transformation and SOC Addition Master Inputs
The following cell requires changing between experimental setups (and, for simplicity for the user, is supposed to be the only part of the protocol that requires any modification).

In [2]:
script_mode = 'simulation' # 'simulation' or 'execution'
experiment_name = 'sTUFPlibrary'
audiofeedback = True # True if you want to to receive audio-notification of a finished process, else change to Falsev

# assembled tubes info
assembly_plate_format = 'opentrons_96_aluminumblock_generic_pcr_strip_200ul' # opentrons_96_aluminumblock_biorad_wellplate_200ul
assembly_plate_column_usage = 'all' # allows you to selectively only use 'odd'- or 'even'-numbered columns or 'all'; useful for 0.2 ml tube usage
# competent cell info
comp_cell_plate_format = 'opentrons_96_aluminumblock_generic_pcr_strip_200ul'
comp_cell_plate_column_usage = 'all' 
comp_cell_fill_start_position_well = 'A1'
# SOC info
SOC_rack_format = 'opentrons_6_tuberack_falcon_50ml_conical'
SOC_position = 'B3'
SOC_vol_per_well = 125.0 # units: µl
# choose what chem. comp. cells you own & what cell volume you want to perform the transformation on
vol_of_purchased_comp_cell_tube = 50
comp_cell_vol_per_assembly = 12.

# 2- Import Statements

In [3]:
import numpy as np
import itertools
import pandas as pd
from IPython.display import display, clear_output, Audio, display, display_html
from copy import deepcopy
import time
from datetime import date
import math
from copy import deepcopy
import os

from iBioFoundry_helper import *
from opentrons import protocol_api, execute
from opentrons import simulate

In [4]:
# establish sole Jupyter Notebook control 
if script_mode == 'execution':
    try:
        os.system("systemctl stop opentrons-robot-server")
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    except:
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in exectution mode!\nYou should see the lights turn on and hear the gantry homing!")
elif script_mode == 'simulation':
    protocol = simulate.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in simulation mode!")
protocol.set_rail_lights(True)
protocol.home()

/Users/camillomoschner/.opentrons/robot_settings.json not found. Loading defaults
/Users/camillomoschner/.opentrons/deck_calibration.json not found. Loading defaults


You are operating this script in simulation mode!


# 3- Function Definitions

In [5]:
def smart_transfer_liquid(transfer_vol, aspiration_pos,dispensation_pos,
                          pick_up_tip=True, touch_tip_before=True,
                          cyclical_mix_before=None,mix_before_reps=2,mix_rate=0.5,
                          asp_bot_clearance=1.5,dispens_bot_clearance=1.5,
                          asp_rate=0.8,disp_rate=0.5,
                          cyclical_mix_after=None,mix_after=None,
                          dispens_top_offset=-2, blow_out=False, air_gap=False, touch_tip=True, touch_miniscus=None,
                          drop_tip=True):
    """
    Liquid transfer that automatically identifies which tip to use.
    """
    if (transfer_vol >= p20_left.min_volume) & (transfer_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (transfer_vol > p300_right.min_volume) & (transfer_vol <= p300_right.max_volume):
        pipette_to_use = p300_right
    else:
        print(f"WARNING: transfer volume not manageable")
    if pick_up_tip==True:
        pipette_to_use.pick_up_tip()
    if cyclical_mix_before != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_before[0]):
            pipette_to_use.aspirate(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[2]))
            pipette_to_use.dispense(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[3]))
    if mix_before_reps != 0:
        pipette_to_use.mix(mix_before_reps,transfer_vol*0.8,aspiration_pos.bottom(asp_bot_clearance),mix_rate)
    # aspiration
    pipette_to_use.aspirate(transfer_vol,aspiration_pos.bottom(asp_bot_clearance), asp_rate)
    #protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
    pipette_to_use.move_to(aspiration_pos.top())
    #protocol.max_speeds['Z'] = None # reset z axis speed
    if air_gap != False:
        pipette_to_use.air_gap(air_gap)
    protocol.delay(seconds=1)
    if touch_tip_before == True:
        pipette_to_use.touch_tip(v_offset=-2, speed=150)
    # dispensation
    pipette_to_use.dispense(transfer_vol,dispensation_pos.bottom(dispens_bot_clearance),disp_rate)
    if cyclical_mix_after != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_after[0]):
            pipette_to_use.aspirate(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[2]))
            pipette_to_use.dispense(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[3]))
    if mix_after != None:
        pipette_to_use.mix(mix_after[0],mix_after[1], dispensation_pos.bottom(mix_after[2]),mix_rate)
    pipette_to_use.move_to(dispensation_pos.top())
    protocol.delay(seconds=2)
    if blow_out == True:
        pipette_to_use.blow_out(dispensation_pos.top(dispens_top_offset))
    if touch_tip == True:
        pipette_to_use.touch_tip(v_offset=dispens_top_offset, speed=150)
    if touch_miniscus != None:
        protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
        pipette_to_use.move_to(dispensation_pos.bottom(touch_miniscus))
        protocol.max_speeds['Z'] = None # reset z axis speed
    if drop_tip==True:
        pipette_to_use.drop_tip()

def smart_distribute_liquid(single_distr_vol, aspiration_pos,dispensation_pos_list,
                            asp_bot_clearance=1.1,dispens_bot_clearance=1.8,
                            asp_flow_rate = 1,disp_flow_rate = 1,
                            touch_tip_bool=True):
    """
    Liquid distribution from one aspiration well into multiple destination wells that automatically identifies which pipette to use.
    """
    if (single_distr_vol >= p20_left.min_volume) & (single_distr_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (single_distr_vol > p300_right.min_volume):
        pipette_to_use = p300_right
    # save original/default flow rates
    original_asp_flow_rate = pipette_to_use.flow_rate.aspirate
    original_disp_flow_rate = pipette_to_use.flow_rate.dispense
    # change flow rates if desired
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate *asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate *disp_flow_rate
    # change well_bottom_clearances if desired
    pipette_to_use.well_bottom_clearance.aspirate = asp_bot_clearance
    pipette_to_use.well_bottom_clearance.dispense = dispens_bot_clearance
    # execute
    pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list, touch_tip=touch_tip_bool)
    # set well_bottom_clearances back to default
    pipette_to_use.well_bottom_clearance.dispense = 1
    pipette_to_use.well_bottom_clearance.aspirate = 1
    # set flow rates back to default
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate

---
# 4- Deck assignments

In [6]:
"""Deck creation"""
deck_slot_list =list(np.arange(1,12))
deck_slot_list.append('bin')
deck_slot_df = pd.DataFrame( np.flip(np.array(deck_slot_list).reshape(4,3), axis=0) )

## i. Labware
Resource: [Opentrons Labware Library Webpage](https://labware.opentrons.com/?_gl=1*1iyc1t4*_ga*MTU1MTM1NzU5MS4xNjE2OTczMzY5*_ga_GNSMNLW4RY*MTYzMDYxNzY1NS4yMC4wLjE2MzA2MTc2NTUuMA..&_ga=2.110416899.767557364.1630617656-1551357591.1616973369)
### a. Modules

In [7]:
temp_mod_name  = 'temperature module gen2'
temp_mod = protocol.load_module(temp_mod_name,'3')
deck_slot_df.iloc[3,2] = temp_mod_name

### b. Racks & Plates

In [8]:
tiprack20a = protocol.load_labware('opentrons_96_tiprack_20ul','8')
tiprack20b = protocol.load_labware('opentrons_96_tiprack_20ul','11')
tiprack20c = protocol.load_labware('opentrons_96_tiprack_20ul','7')
tiprack300a = protocol.load_labware('opentrons_96_tiprack_300ul','10')
# tuberacks & plates
assembly_plate = protocol.load_labware(assembly_plate_format,'2') # opentrons_96_aluminumblock_biorad_wellplate_200ul
comp_cell_plate = temp_mod.load_labware(comp_cell_plate_format,'3') # opentrons_24_aluminumblock_nest_1.5ml_screwcap
SOC_rack = protocol.load_labware(SOC_rack_format,'4') 

In [9]:
labware_list = [tiprack20a,tiprack20b,tiprack20c, tiprack300a,
                assembly_plate, comp_cell_plate,SOC_rack]
_ =[update_labware(deck_slot_df, labware_item) for labware_item in labware_list]
# visually inspect deck
print("\nFinal OT-2 Deck Configuration:");(deck_slot_df)


Final OT-2 Deck Configuration:


,0,1,2
0,10-opentrons_96_tiprack_300ul,11-opentrons_96_tiprack_20ul,bin
1,7-opentrons_96_tiprack_20ul,8-opentrons_96_tiprack_20ul,9
2,4-opentrons_6_tuberack_falcon_50ml_conical,5,6
3,1,2-opentrons_96_aluminumblock_generic_pcr_strip...,temperature module gen2


## ii. Pipettes

Resource: [Opentrons Pipettes API Webpage](https://docs.opentrons.com/v2/new_pipette.html)

In [10]:
p20_left = protocol.load_instrument('p20_single_gen2','left', tip_racks=[tiprack20a,tiprack20b,tiprack20c])
p300_right = protocol.load_instrument('p300_single_gen2','right', tip_racks=[tiprack300a])

## iii. Assembly info integration

In [11]:
assemblies_info_dir = f"1_GGA{os.path.sep}assemblies_summary.csv"
assemblies_info_df = pd.read_csv(assemblies_info_dir)
assembly_no = len(assemblies_info_df['assembly_ID'])
#combinations_df['cPCR_verified'] = [False for i in np.arange(combinations_no)]
#combinations_df['sequence_verified'] = [False for i in np.arange(combinations_no)]

---
# 5- LH Step Calculations 

In [12]:
assembly_plate_obj = Plate(identify_plate(assembly_plate_format, 'plate_format'),only_columns=assembly_plate_column_usage)
assembly_plate_obj.layout = extend_fill_plate_df_with_list(assembly_plate_obj.layout,assemblies_info_df['assembly_ID'].to_list(),
                                                            fill_start_position=comp_cell_fill_start_position_well, fill_first='rows')
comp_cell_plate_obj = Plate(identify_plate(comp_cell_plate_format, 'plate_format'),only_columns=comp_cell_plate_column_usage)
comp_cell_plate_obj.layout = extend_fill_plate_df_with_list(comp_cell_plate_obj.layout,assemblies_info_df['assembly_ID'].to_list(),
                                                            fill_start_position=comp_cell_fill_start_position_well, fill_first='rows')
# present side-to-side
assembly_plate_styler = assembly_plate_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of Assembly Plate ({assembly_plate.parent})")
comp_cell_plate_styler = comp_cell_plate_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of Competent Cell Plate ({comp_cell_plate.parent})")
space = "\xa0"*10
display_html(assembly_plate_styler._repr_html_()+space+comp_cell_plate_styler._repr_html_() , raw=True)

,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW37,pCMCW38,pCMCW39,pCMCW40,pCMCW41,pCMCW42,pCMCW43,pCMCW44,pCMCW45,pCMCW46,pCMCW47,pCMCW48
B,pCMCW49,pCMCW50,pCMCW51,pCMCW52,pCMCW53,pCMCW54,pCMCW55,pCMCW56,pCMCW57,pCMCW58,pCMCW59,pCMCW60
C,pCMCW61,pCMCW62,pCMCW63,pCMCW64,pCMCW65,pCMCW66,pCMCW67,pCMCW68,pCMCW69,pCMCW70,pCMCW71,pCMCW72
D,pCMCW73,pCMCW74,pCMCW75,pCMCW76,pCMCW77,pCMCW78,pCMCW79,pCMCW80,pCMCW81,pCMCW82,pCMCW83,pCMCW84
E,pCMCW85,pCMCW86,pCMCW87,pCMCW88,pCMCW89,pCMCW90,pCMCW91,pCMCW92,pCMCW93,pCMCW94,pCMCW95,pCMCW96
F,pCMCW97,pCMCW98,pCMCW99,pCMCW100,pCMCW101,pCMCW102,pCMCW103,pCMCW104,pCMCW105,pCMCW106,pCMCW107,pCMCW108
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0
,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW37,pCMCW38,pCMCW39,pCMCW40,pCMCW41,pCMCW42,pCMCW43,pCMCW44,pCMCW45,pCMCW46,pCMCW47,pCMCW48


In [13]:
# calculate how many chemically competent cell tubes you need to get from the -80
no_assemblies_per_cell_tube = math.floor(vol_of_purchased_comp_cell_tube/comp_cell_vol_per_assembly)
print(f"You are thawing chemically competent cells of {vol_of_purchased_comp_cell_tube} µl.\n -> You choose to use {comp_cell_vol_per_assembly} µl chem. comp. cells per transfomration.")
print(f"\n{math.ceil(assembly_no/no_assemblies_per_cell_tube)} ({assembly_no/no_assemblies_per_cell_tube}) chem. competent cell tubes are required!\n")

You are thawing chemically competent cells of 50 µl.
 -> You choose to use 12.0 µl chem. comp. cells per transfomration.

18 (18.0) chem. competent cell tubes are required!



In [14]:
# integrate transformation information into master spreadsheet
transf_plate_positions = [find_df_coordinates(comp_cell_plate_obj.layout,assembly_id) for assembly_id in assemblies_info_df['assembly_ID']]
assemblies_info_df['transformation_tube'] = transf_plate_positions
assemblies_info_df['transformation_date'] = [str(date.today()) for i in range(len(assemblies_info_df))]
assemblies_info_df['SOC_vol'] = [SOC_vol_per_well for i in range(len(assemblies_info_df))]
#assemblies_info_df['colonies'] = [np.nan for i in np.arange(assembly_no)]
#assemblies_info_df['white_colonies'] = [np.nan for i in np.arange(assembly_no)]
print(f"\n -> {assemblies_info_df.SOC_vol.sum()} µl of SOC medium is required for these assembly! Put that volume into a 50 ml Falcon tube in rack position {SOC_position}.\n")
assemblies_info_df


 -> 9000.0 µl of SOC medium is required for these assembly! Put that volume into a 50 ml Falcon tube in rack position B3.



,assembly_ID,assembly_description,Part 1: Promoter,Part 2: RBS,Part 3: CDS,Part 4: Terminator,Part 5: LVL1 Destination Vector,inter_stock_ID,assembly_tube,assembly_date,transformation_tube,transformation_date,SOC_vol,CP_plate_format,CP_plate_no,CP_pos_per_round,CP_wells,colonies
0,pCMCW37,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,A1,09/05/2022,A1,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,0,0,['A3'],NaN
1,pCMCW38,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,B1,09/05/2022,A2,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,0,0,['A2'],NaN
2,pCMCW39,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,C1,09/05/2022,A3,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,0,0,['A1'],NaN
3,pCMCW40,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,D1,09/05/2022,A4,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,0,0,['B3'],NaN
4,pCMCW41,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0034,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0034|CIDAR_LVL0_CDS_mVen...,E1,09/05/2022,A5,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,0,0,['B2'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,pCMCW104,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0032...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,B12,09/05/2022,F8,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,11,1,['A2'],NaN
68,pCMCW105,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0033...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,C12,09/05/2022,F9,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,11,1,['A1'],NaN
69,pCMCW106,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0033...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,D12,09/05/2022,F10,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,11,1,['B3'],NaN
70,pCMCW107,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0034...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0034,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0034|CIDAR_LVL0_CDS_mVen...,E12,09/05/2022,F11,2022-05-10,125.0,corning_6_wellplate_16.8ml_flat,11,1,['B2'],NaN


## Update summary Spreadsheets

In [19]:
update_assembly_summary_answer = input("Do you want to update the GGA summary spreadsheet? (yes / no)\n answer:")
if update_assembly_summary_answer == 'yes':
    assemblies_info_df.to_csv(assemblies_info_dir,index=False)
    print(f"\n -> Your GGA summary spreadsheet for experiment \'{experiment_name}\' has been updated!\n")
else:
    print(f"\n -> Ensure proper recording of your results!\n")

Do you want to update the GGA summary spreadsheet? (yes / no)
 answer: no



 -> Ensure proper recording of your results!



In [16]:
temp_mod.set_temperature(6)
if audiofeedback == True:
    allDone()

---
# 6- Execution
## i. Transfer assemblies to cells

In [17]:
print("Assembly transfer...")
transformation_vol = 2.0 # units" µl
for idx, current_assembly_info in enumerate([assemblies_info_df.iloc[row,:] for row in range(len(assemblies_info_df))]):
    print(f"  {idx } - {transformation_vol} µl {current_assembly_info['assembly_ID']}:\n             assembly_plate \'{current_assembly_info['assembly_tube']}\' -> comp_cells_plate \'{current_assembly_info['transformation_tube']}\'")
    smart_transfer_liquid(transformation_vol,
                              assembly_plate[ current_assembly_info['assembly_tube'] ],
                              comp_cell_plate[ current_assembly_info['transformation_tube'] ],
                              asp_bot_clearance=1,asp_rate=0.2, touch_tip_before=False,touch_tip=False,
                              blow_out=True,touch_miniscus=2.5,
                              dispens_bot_clearance=2.5,disp_rate=0.1)
if audiofeedback == True:
    allDone()

Assembly transfer...
  0 - 2.0 µl pCMCW37:
             assembly_plate 'A1' -> comp_cells_plate 'A1'
  1 - 2.0 µl pCMCW38:
             assembly_plate 'B1' -> comp_cells_plate 'A2'
  2 - 2.0 µl pCMCW39:
             assembly_plate 'C1' -> comp_cells_plate 'A3'
  3 - 2.0 µl pCMCW40:
             assembly_plate 'D1' -> comp_cells_plate 'A4'
  4 - 2.0 µl pCMCW41:
             assembly_plate 'E1' -> comp_cells_plate 'A5'
  5 - 2.0 µl pCMCW42:
             assembly_plate 'F1' -> comp_cells_plate 'A6'
  6 - 2.0 µl pCMCW43:
             assembly_plate 'A2' -> comp_cells_plate 'A7'
  7 - 2.0 µl pCMCW44:
             assembly_plate 'B2' -> comp_cells_plate 'A8'
  8 - 2.0 µl pCMCW45:
             assembly_plate 'C2' -> comp_cells_plate 'A9'
  9 - 2.0 µl pCMCW46:
             assembly_plate 'D2' -> comp_cells_plate 'A10'
  10 - 2.0 µl pCMCW47:
             assembly_plate 'E2' -> comp_cells_plate 'A11'
  11 - 2.0 µl pCMCW48:
             assembly_plate 'F2' -> comp_cells_plate 'A12'
  12 - 2.0 µl 

In [18]:
# deactivate temperature module
temp_mod.deactivate()
protocol.home()

## iii. Chem. Transformation in Thermocycler

## ii. Transfer SOC to cells

In [20]:
print("SOC transfer...")
print(f" ... {SOC_vol_per_well} µl of cell recovery medium distributed into:\n   {assemblies_info_df['transformation_tube'].to_list()[:int(assembly_no/2)]}\n   {assemblies_info_df['transformation_tube'].to_list()[int(assembly_no/2):]}")
smart_distribute_liquid(SOC_vol_per_well, SOC_rack[SOC_position], 
                        [comp_cell_plate[well] for well in assemblies_info_df['transformation_tube'].to_list()],
                        asp_bot_clearance=2,dispens_bot_clearance=18,
                        asp_flow_rate = 1.5,disp_flow_rate = 5,
                        touch_tip_bool=False)
if audiofeedback == True:
    allDone()

SOC transfer...
 ... 125.0 µl of cell recovery medium distributed into:
   ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12']
   ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12']


# Shutdown

In [20]:
# allDone()
try:
    p20_left.drop_tip()
except:
    pass
try:
    p300_right.drop_tip()
except:
    pass
temp_mod.deactivate()
protocol.home()

In [21]:
# complete tip usage check:
_=[display(show_rack_usage(labware)[1]) for labware in labware_list if 'Opentrons 96 Tip Rack' in str(labware)]

Opentrons 96 Tip Rack 20 µL on 8:
 -> 72 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,x,x,x,x,x,x,x,x,0,0,0
B,x,x,x,x,x,x,x,x,x,0,0,0
C,x,x,x,x,x,x,x,x,x,0,0,0
D,x,x,x,x,x,x,x,x,x,0,0,0
E,x,x,x,x,x,x,x,x,x,0,0,0
F,x,x,x,x,x,x,x,x,x,0,0,0
G,x,x,x,x,x,x,x,x,x,0,0,0
H,x,x,x,x,x,x,x,x,x,0,0,0


Opentrons 96 Tip Rack 20 µL on 11:
 -> 0 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,0,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 20 µL on 7:
 -> 0 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,0,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 300 µL on 10:
 -> 72 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,x,x,x,x,x,x,x,x,0,0,0
B,x,x,x,x,x,x,x,x,x,0,0,0
C,x,x,x,x,x,x,x,x,x,0,0,0
D,x,x,x,x,x,x,x,x,x,0,0,0
E,x,x,x,x,x,x,x,x,x,0,0,0
F,x,x,x,x,x,x,x,x,x,0,0,0
G,x,x,x,x,x,x,x,x,x,0,0,0
H,x,x,x,x,x,x,x,x,x,0,0,0
